In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
import networkx as nx

In [101]:
df_grid = pd.read_excel('./grid.xlsx', index_col=0)
df_grid_original_lines = pd.read_excel('../20230301_Core Static Grid Model_3.xlsx', sheet_name='Lines', index_col=0)
df_grid_original_tielines = pd.read_excel('../20230301_Core Static Grid Model_3.xlsx', sheet_name='Tielines', index_col=0)

columns = [
    'MaximumCurrentImax(A)',
    'Unnamed: 7',
    'Unnamed: 8',
    'Unnamed: 9',
    'Unnamed: 10',
    'Unnamed: 11',
    'Unnamed: 12',
    'Dynamiclinerating(DLR)',
    'Unnamed: 14',
]

for index, row in df_grid_original_lines.iterrows():
    for column in columns:
        if isinstance(row[column], str):
            df_grid_original_lines.loc[index, column] = np.nan
            
for index, row in df_grid_original_tielines.iterrows():
    for column in columns:
        if isinstance(row[column], str):
            df_grid_original_tielines.loc[index, column] = np.nan

In [102]:
display(df_grid_original_lines)

,Unnamed: 1,Unnamed: 2,Substation_1,Substation_2,Unnamed: 5,MaximumCurrentImax(A),Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Dynamiclinerating(DLR),Unnamed: 14,Electrical Parameters,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
NE_name,EIC_Code,TSO,Full_name,Full_name,Voltage_level(kV),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Resistance_R(Ω),Reactance_X(Ω),Susceptance_B(μS),Length_(km),Comment
Eula - Röhrsdorf 203,11TD8L203------C,50HERTZ,Eula,Röhrsdorf,220,790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0937,18.349,161.163703,51.511,NaN
Röhrsdorf - Weida 207,11TD8L207------H,50HERTZ,Röhrsdorf,Weida,220,790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.5895,31.888,348.716785,98.324,NaN
Eula - Weida 208,11TD8L208------9,50HERTZ,Eula,Weida,220,1070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0537,19.541,264.239358,67.55,NaN
Bentwisch - Güstrow 275,11TD8L275------7,50HERTZ,Bentwisch,Güstrow,220,1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.909,12.055,158.084942,41.418,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y-Stokach (-Gurtweil - Laufenburg) ge-rt,11T0-0000-1626-3,TRANSNETBW,Y-Gurtweil,Stockach,220,NaN,NaN,NaN,NaN,NaN,NaN,1500,NaN,NaN,2.1,16.8,347.3659,69.71,tripod - length until Y-Node
Y-Villingen (-Engstlatt - Kuehmoss) gn,11TD400000587-3M,TRANSNETBW,Y-Engstlatt,Villingen,400,NaN,NaN,NaN,NaN,NaN,NaN,478,NaN,NaN,0.45,3.82,78.22566,16.2,tripod - length until Y-Node
Y-Weier (-Daxlanden - Kuppenheim) ge,11TD40000A574-36,TRANSNETBW,Y-Daxlanden,Weier,220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2524,2797,1.4,10.75,230.6872,46.71,tripod - length until Y-Node
Y-Weinheim (-GKM - Neurott) rt,11T0-0000-1640-D,TRANSNETBW,GKM,Weinheim,220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2505,3000,0.1,0.2,1.57,0.12,tripod - length until Y-Node


In [103]:
for index, row in df_grid.iterrows():
    query = df_grid_original_lines[df_grid_original_lines['Unnamed: 1'] == row['eic']]
    if len(query) > 0:
        print( query.iloc[0][[
            'MaximumCurrentImax(A)',
            'Unnamed: 7',
            'Unnamed: 8',
            'Unnamed: 9',
            'Unnamed: 10',
            'Unnamed: 11',
            'Unnamed: 12',
            'Dynamiclinerating(DLR)',
            'Unnamed: 14',
        ]])
        df_grid.loc[index, 'max_current'] = query.iloc[0][[
            'MaximumCurrentImax(A)',
            'Unnamed: 7',
            'Unnamed: 8',
            'Unnamed: 9',
            'Unnamed: 10',
            'Unnamed: 11',
            'Unnamed: 12',
            'Dynamiclinerating(DLR)',
            'Unnamed: 14',
        ]].mean()
    else:
        query = df_grid_original_tielines[df_grid_original_tielines['Unnamed: 1'] == row['eic']]
        if len(query) > 0:
            df_grid.loc[index, 'max_current'] = query.iloc[0][[
                'MaximumCurrentImax(A)',
                'Unnamed: 7',
                'Unnamed: 8',
                'Unnamed: 9',
                'Unnamed: 10',
                'Unnamed: 11',
                'Unnamed: 12',
                'Dynamiclinerating(DLR)',
                'Unnamed: 14',
            ]].mean()

MaximumCurrentImax(A)     790
Unnamed: 7                NaN
Unnamed: 8                NaN
Unnamed: 9                NaN
Unnamed: 10               NaN
Unnamed: 11               NaN
Unnamed: 12               NaN
Dynamiclinerating(DLR)    NaN
Unnamed: 14               NaN
Name: Eula - Röhrsdorf 203, dtype: object
MaximumCurrentImax(A)     790
Unnamed: 7                NaN
Unnamed: 8                NaN
Unnamed: 9                NaN
Unnamed: 10               NaN
Unnamed: 11               NaN
Unnamed: 12               NaN
Dynamiclinerating(DLR)    NaN
Unnamed: 14               NaN
Name: Röhrsdorf - Weida 207, dtype: object
MaximumCurrentImax(A)     1070
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: Eula - Weida 208, dtype: object
MaximumCurrentImax(A)     1200
Unnamed: 7                 

MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)     740
Unnamed: 14               1031
Name: Y-Ensdorf (-Diefflen - Uchtelfangen) ENSDF S , dtype: object
MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12               1360
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: Y-Diefflen (-Ensdorf - Uchtelfangen) ENSDF S , dtype: object
MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12               1360
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: Y-Uchtelfangen (-Dieffl

MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12               2580
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: Niederrhein - Utfort ZENSEN S , dtype: object
MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12               1360
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: Osterath - St. Peter ZONS O , dtype: object
MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)     NaN
Unnamed: 14               1548
Name: Norf - Osterath ZONS W , dtype: object
MaximumCurrentIm

MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12               1920
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: Maribor - Cirkovce, dtype: object
MaximumCurrentImax(A)     NaN
Unnamed: 7                NaN
Unnamed: 8                NaN
Unnamed: 9                NaN
Unnamed: 10               NaN
Unnamed: 11               NaN
Unnamed: 12               960
Dynamiclinerating(DLR)    NaN
Unnamed: 14               NaN
Name: Podlog - Bericevo, dtype: object
MaximumCurrentImax(A)     2448.32
Unnamed: 7                2317.16
Unnamed: 8                   2186
Unnamed: 9                2317.16
Unnamed: 10               2448.32
Unnamed: 11                   NaN
Unnamed: 12                   NaN
Dynamiclinerating(DLR)        NaN
Unnamed: 14                   NaN
Name: AUBANGE – BRUME 380.13, dtype: object
MaximumCurrentImax(A) 

MaximumCurrentImax(A)       2000
Unnamed: 7                  2000
Unnamed: 8                1900.7
Unnamed: 9                  2000
Unnamed: 10                  NaN
Unnamed: 11                  NaN
Unnamed: 12                  NaN
Dynamiclinerating(DLR)      1610
Unnamed: 14                 2000
Name: TUCZNAWA-ROGOWIEC, dtype: object
MaximumCurrentImax(A)       2000
Unnamed: 7                1894.4
Unnamed: 8                1779.6
Unnamed: 9                1894.4
Unnamed: 10                  NaN
Unnamed: 11                  NaN
Unnamed: 12                  NaN
Dynamiclinerating(DLR)      1470
Unnamed: 14                 2000
Name: GRUDZIADZ-PLOCK, dtype: object
MaximumCurrentImax(A)     1200
Unnamed: 7                1200
Unnamed: 8                1200
Unnamed: 9                1200
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)     950
Unnamed: 14               1200
Name: RZESZOW-TARNOW, dtype: object
MaximumCurrentI

...

MaximumCurrentImax(A)     1308
Unnamed: 7                1408
Unnamed: 8                1408
Unnamed: 9                1478
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: LIT 225kV N0 1 PORT-JEROME-SANDOUVILLE, dtype: object
MaximumCurrentImax(A)     618
Unnamed: 7                745
Unnamed: 8                745
Unnamed: 9                819
Unnamed: 10               NaN
Unnamed: 11               NaN
Unnamed: 12               NaN
Dynamiclinerating(DLR)    NaN
Unnamed: 14               NaN
Name: LIT 225kV N0 1 PORT-JEROME-VAUPALIERE (LA), dtype: object
MaximumCurrentImax(A)     1280
Unnamed: 7                1332
Unnamed: 8                1332
Unnamed: 9                1382
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: LIT 225kV N0 1 POTEAU-ROUGE - SQUIVIDAN, d

MaximumCurrentImax(A)     NaN
Unnamed: 7                NaN
Unnamed: 8                NaN
Unnamed: 9                NaN
Unnamed: 10               NaN
Unnamed: 11               NaN
Unnamed: 12               785
Dynamiclinerating(DLR)    NaN
Unnamed: 14               NaN
Name: Sala - Krizovany, dtype: object
MaximumCurrentImax(A)     NaN
Unnamed: 7                NaN
Unnamed: 8                NaN
Unnamed: 9                NaN
Unnamed: 10               NaN
Unnamed: 11               NaN
Unnamed: 12               360
Dynamiclinerating(DLR)    NaN
Unnamed: 14               NaN
Name: Siroka - Sucany, dtype: object
MaximumCurrentImax(A)     NaN
Unnamed: 7                NaN
Unnamed: 8                NaN
Unnamed: 9                NaN
Unnamed: 10               NaN
Unnamed: 11               NaN
Unnamed: 12               360
Dynamiclinerating(DLR)    NaN
Unnamed: 14               NaN
Name: Siroka - Sucany, dtype: object
MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8      

MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)     918
Unnamed: 14               1310
Name: Bechterdissen - Veltheim A01, dtype: object
MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12               1310
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: Bechterdissen - Lage A03, dtype: object
MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)     780
Unnamed: 14               1108
Name: Veltheim - Bechterdissen A07, dtype: object
MaximumCurrentIma

MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)    2216
Unnamed: 14               3150
Name: Ovenstaedt - Sottrum 3, dtype: object
MaximumCurrentImax(A)     NaN
Unnamed: 7                NaN
Unnamed: 8                NaN
Unnamed: 9                NaN
Unnamed: 10               NaN
Unnamed: 11               NaN
Unnamed: 12               816
Dynamiclinerating(DLR)    NaN
Unnamed: 14               NaN
Name: Y-Plattling (-Plattling - Pleinting - Schwandorf) 465, dtype: object
MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12               2204
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: Y-Plattling (-Pleinting - Schwandorf) 465, dtyp

MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)    2619
Unnamed: 14               3600
Name: Redwitz - Remptendorf 413, dtype: object
MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12                NaN
Dynamiclinerating(DLR)    2619
Unnamed: 14               3600
Name: Redwitz - Remptendorf 414, dtype: object
MaximumCurrentImax(A)      NaN
Unnamed: 7                 NaN
Unnamed: 8                 NaN
Unnamed: 9                 NaN
Unnamed: 10                NaN
Unnamed: 11                NaN
Unnamed: 12               2748
Dynamiclinerating(DLR)     NaN
Unnamed: 14                NaN
Name: Mecklar - Vieselbach 449, dtype: object
MaximumCurrentImax(A)  

In [106]:
df_grid.to_excel('./grid.xlsx')

In [108]:
df_grid.max_current.mean()

1852.19258239656